In [1]:
import pandas as pd
from pyspark.sql.functions import udf, col
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, mean
from pyspark.sql.types import *

# This notebook is for feature measurement 

sc = pyspark.SparkContext()

path_x = '/Users/tiegsti.solomon/Downloads'
path_y = '/Users/tiegsti.solomon'

events = sc.textFile(path_x+"/events.csv")
doc_topics = sc.textFile(path_y+"/nodes_recovered.csv")

In [2]:
# remove header train
rdd_events_head = events.filter(lambda l: "display_id" in l)
rdd_events = events.subtract(rdd_events_head)

# remove header test
rdd_topics_head = doc_topics.filter(lambda l: "uuid" in l)
rdd_topics = doc_topics.subtract(rdd_topics_head)

rdd_e = rdd_events.map(lambda a: a.split(","))
rdd_t = rdd_topics.map(lambda a: a.split(","))

sqlContext = SQLContext(sc)
events_ = sqlContext.createDataFrame(rdd_e) # schema err
topics_ = sqlContext.createDataFrame(rdd_t)
topics_.show(5)
events_.show(5)

events_dfp = events_.toPandas()
topics_dfp = topics_.toPandas()

+------+---+---------+----------+
|    _1| _2|       _3|        _4|
+------+---+---------+----------+
| 11395|  0| 37.09024|-95.712891|
|280649|  1| 37.09024|-95.712891|
|  6923|  0| 37.09024|-95.712891|
|175534|  0|-0.023559| 37.906193|
|221749|  1| 37.09024|-95.712891|
+------+---+---------+----------+
only showing top 5 rows

+-------+--------------+-------+---------+---+---------+
|     _1|            _2|     _3|       _4| _5|       _6|
+-------+--------------+-------+---------+---+---------+
|3690712|b1fd1d05c4b66a|1027644|236206265|  2|US>NJ>504|
|9745476|e4633fc85ef2a1|2140078|656541359|  1|US>TN>557|
|6263905|2d7d69b1aaa032|2181607|418207638|  2|US>IL>610|
|4266820|f745eda9d0ea54|2011906|280124441|  2|       PH|
|8006793|9b9043346efd56|1788292|554591670|  1|US>FL>528|
+-------+--------------+-------+---------+---+---------+
only showing top 5 rows



In [3]:
def convert_id_list(doc_ids):
    return ' '.join(d.split(':')[1] for d in doc_ids.split(' '))

events_dfp = events_dfp['_3'].unique()
doc_ids = pd.DataFrame(events_dfp, columns = ['_3'])
doc_ids = doc_ids.rename(index=str, columns={'_3':'_1'})
topic_ids = pd.DataFrame(topics_dfp, columns = ['_1','_2'])

merged_dfp = doc_ids.merge(topic_ids, how='right')

In [4]:
doc_ids

,_1
0,1027644
1,2140078
2,2181607
3,2011906
4,1788292
5,435161
6,1569589
7,2115948
8,2245558
9,2320484
